In [2]:
# --- Install required libraries ---
!pip install PyPDF2 textblob pandas

# --- Import necessary libraries ---
import os
import re
from collections import defaultdict
import PyPDF2
from textblob import TextBlob
import pandas as pd
import json

# --- Function: Extract text from PDF ---
def extract_text_from_pdf(file_path):
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        return None
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            full_text = []
            for page in pdf_reader.pages:
                text = page.extract_text()
                if text:
                    full_text.append(text)
        return "\n".join(full_text)
    except Exception as e:
        print(f"💥 Error reading {file_path}: {e}")
        return None

# --- Function: Analyze sentiment ---
def analyze_sentiment(text):
    if not text:
        return {"polarity": None, "subjectivity": None}
    blob = TextBlob(text)
    return {
        "polarity": blob.sentiment.polarity,
        "subjectivity": blob.sentiment.subjectivity,
    }

# 🔁 ĐÂY LÀ DÒNG BẠN CẦN ĐỔI FOLDER:
base_dir = "/Users/mynhie/Desktop/Thesis/Fall 2025 version/data done/X3 0611/PDF_exports_all"

# Get list of all PDF files
all_pdfs = [f for f in os.listdir(base_dir) if f.endswith(".pdf")]

# Function: Extract subreddit base name
def extract_base_subreddit(filename):
    name = re.sub(r"^r_", "", filename)
    name = re.sub(r"_\d{4}.*\.pdf$", "", name)
    name = re.sub(r"\.pdf$", "", name)
    # Split by space, underscore, or dash
    parts = re.split(r"[ _-]+", name)
    base = parts[0].lower()
    # Handle AI/A.I/Artificial cases → r/ai
    if re.search(r"^a(\.?i|rtificial)", base):
        base = "ai"
    return f"r/{base}"

# Group all PDFs by subreddit
article_paths = defaultdict(list)
for f in all_pdfs:
    full_path = os.path.join(base_dir, f)
    base_key = extract_base_subreddit(f)
    article_paths[base_key].append(full_path)

# Show grouping summary
print(f"📂 Found {len(all_pdfs)} PDF files across {len(article_paths)} subreddits.")
for sub, files in sorted(article_paths.items()):
    print(f"{sub}: {len(files)} file(s)")

# Save mapping to JSON
with open("article_paths_grouped_all.json", "w") as f:
    json.dump(article_paths, f, indent=2)
print("\n✅ Saved mapping to article_paths_grouped_all.json")

# --- Function: Process all articles & calculate sentiment ---
def process_selected_articles(selected_sources):
    results = []
    total_files = sum(len(v) for v in selected_sources.values())
    processed = 0
    for source, paths in selected_sources.items():
        for file_path in paths:
            processed += 1
            print(f"📄 [{processed}/{total_files}] Processing: {file_path}")
            text = extract_text_from_pdf(file_path)
            sentiment = analyze_sentiment(text)
            sentiment.update({"Source": source, "File": file_path})
            results.append(sentiment)
    return results

# --- 🔍 Analyze ALL subreddits ---
selected_sources = dict(article_paths)  # all 215 files grouped by subreddit

# Run full sentiment analysis
results = process_selected_articles(selected_sources)
df = pd.DataFrame(results)

# --- 📊 Average sentiment across all articles ---
average_sentiment_overall = df.dropna()[["polarity", "subjectivity"]].mean()
print("\n📈 Average Sentiment across ALL 215 PDFs:")
print(average_sentiment_overall)

# --- 📂 Average sentiment by subreddit group ---
average_sentiment_by_group = df.dropna().groupby("Source")[["polarity", "subjectivity"]].mean().sort_values(by="polarity")
print("\n📊 Average Sentiment by Subreddit:")
print(average_sentiment_by_group)

# --- 🔝 Top 10 most extreme sentiment files ---
df["absolute_polarity"] = df["polarity"].abs()
top_10_articles = df.sort_values(by="absolute_polarity", ascending=False).head(10)
print("\n🔥 Top 10 articles with highest absolute polarity:")
print(top_10_articles[["polarity", "absolute_polarity", "Source", "File"]])

# --- 💾 Save results ---
df.to_csv("sentiment_analysis_all.csv", index=False)
print("\n✅ Saved results to sentiment_analysis_all.csv")

# --- Display summary ---
print(f"\n🎉 Sentiment analysis completed for {len(df)} PDF files across {len(article_paths)} subreddits.")

📂 Found 1199 PDF files across 82 subreddits.
r/acne: 4 file(s)
r/ai: 50 file(s)
r/anime: 9 file(s)
r/animeart: 14 file(s)
r/animesketch: 14 file(s)
r/anticonsumption: 18 file(s)
r/askscience: 20 file(s)
r/autos: 10 file(s)
r/baldursgate3: 15 file(s)
r/batman: 32 file(s)
r/beamazed: 9 file(s)
r/beauty: 5 file(s)
r/biology: 14 file(s)
r/bobsburgers: 12 file(s)
r/booksuggestions: 13 file(s)
r/bravorealhousewives: 11 file(s)
r/characterai: 14 file(s)
r/colorization: 43 file(s)
r/cooking: 50 file(s)
r/corgi: 9 file(s)
r/creepy: 17 file(s)
r/cryptotechnology: 8 file(s)
r/cycling: 7 file(s)
r/designmyroom: 12 file(s)
r/developersindia: 22 file(s)
r/disney: 14 file(s)
r/diy: 9 file(s)
r/dndbehindthescreen: 15 file(s)
r/documentaries: 13 file(s)
r/dota2: 14 file(s)
r/economy: 8 file(s)
r/food: 10 file(s)
r/funny: 11 file(s)
r/gamedev: 11 file(s)
r/gamephysics: 12 file(s)
r/germany: 14 file(s)
r/goldenretrievers: 9 file(s)
r/gpt3: 33 file(s)
r/gravityfalls: 12 file(s)
r/harrypotter: 14 file(s)
r